In [1]:
from __future__ import print_function

from miscc.utils import mkdir_p
from miscc.utils import build_super_images
from miscc.losses import sent_loss, words_loss
from miscc.config import cfg, cfg_from_file

from datasets import TextDataset
from datasets import prepare_data

from model import RNN_ENCODER, CNN_ENCODER

import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import argparse
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

In [2]:
def build_models():
    # build model ############################################################
    text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    print("cfg.TEXT.EMBEDDING_DIM", cfg.TEXT.EMBEDDING_DIM) #256
#     image_encoder = CNN_ENCODER(cfg.TEXT.EMBEDDING_DIM)
    labels = Variable(torch.LongTensor(range(batch_size)))
    start_epoch = 0
    if cfg.TRAIN.NET_E != '':
        state_dict = torch.load(cfg.TRAIN.NET_E)
        text_encoder.load_state_dict(state_dict)
        print('Load ', cfg.TRAIN.NET_E)
        #
        name = cfg.TRAIN.NET_E.replace('text_encoder')
        state_dict = torch.load(name)
#         image_encoder.load_state_dict(state_dict)
        print('Load ', name)

        istart = cfg.TRAIN.NET_E.rfind('_') + 8
        iend = cfg.TRAIN.NET_E.rfind('.')
        start_epoch = cfg.TRAIN.NET_E[istart:iend]
        start_epoch = int(start_epoch) + 1
        print('start_epoch', start_epoch)
    if cfg.CUDA:
        text_encoder = text_encoder
#         image_encoder = image_encoder
        labels = labels

    return text_encoder, labels, start_epoch

In [3]:
print(torch.__version__)

0.4.0


In [4]:
#Configurations
cfg_from_file("cfg/DAMSM/coco.yml")
import pprint
pprint.pprint(cfg)
random.seed(random.randint(1, 10000))
np.random.seed(random.randint(1, 10000))
torch.manual_seed(random.randint(1, 10000))
if cfg.CUDA:
    torch.cuda.manual_seed_all(random.randint(1, 10000))
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = '../output/%s_%s_%s' % \
    (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

model_dir = os.path.join(output_dir, 'Model')
image_dir = os.path.join(output_dir, 'Image')
mkdir_p(model_dir)
mkdir_p(image_dir)
# Debug
torch.cuda.device(cfg.GPU_ID)
cudnn.benchmark = True

{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'coco',
 'DATA_DIR': '../data/coco',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 5, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 15},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 5},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}


In [5]:
# cfg.TREE.BASE_SIZE = 299
imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM-1))
batch_size = cfg.TRAIN.BATCH_SIZE
image_transform = transforms.Compose([
    transforms.Scale(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])
dataset = TextDataset("../data/coco", 'train',
                      base_size=299,
                      transform=image_transform)
print("=====================================================")
print("cfg.DATA_DIR: ", cfg.DATA_DIR)
print("cfg.TREE.BASE_SIZE: ", cfg.TREE.BASE_SIZE)
print("image_transform: ", image_transform)
print("cfg.TEXT.EMBEDDING_DIM", cfg.TEXT.EMBEDDING_DIM)
print("=====================================================")
print("dataset.n_words:", dataset.n_words, " dataset.embeddings_num: ", dataset.embeddings_num)
assert dataset
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

# # validation data #
dataset_val = TextDataset("../data/coco", 'test',
                          base_size=cfg.TREE.BASE_SIZE,
                          transform=image_transform)
dataloader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle
cfg.DATA_DIR:  ../data/coco
cfg.TREE.BASE_SIZE:  299
image_transform:  <torchvision.transforms.Compose object at 0x1c14804128>
cfg.TEXT.EMBEDDING_DIM 256
dataset.n_words: 27297  dataset.embeddings_num:  5
Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle


In [7]:
dataset

In [8]:
text_encoder, labels, start_epoch = build_models()

cfg.TEXT.EMBEDDING_DIM 256


/Users/user/anaconda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [9]:
text_encoder.train()
text_encoder.eval()

RNN_ENCODER(
  (encoder): Embedding(27297, 300)
  (drop): Dropout(p=0.5)
  (rnn): LSTM(300, 128, batch_first=True, dropout=0.5, bidirectional=True)
)

In [10]:
for step, data in enumerate(dataloader, 0):
    imgs, captions, cap_lens, class_ids, keys = prepare_data(data)
    hidden = text_encoder.init_hidden(batch_size)
    print("step", step, "\ncap_lens\n", cap_lens, "\n") 
    if step == 1:
        break

step 0 
cap_lens
 tensor([ 15,  15,  15,  15,  14,  13,  13,  12,  11,  11,  11,  11,
         11,  11,  11,  11,  11,  10,  10,  10,  10,  10,  10,  10,
         10,  10,  10,   9,   9,   9,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   8]) 

step 1 
cap_lens
 tensor([ 15,  15,  15,  15,  13,  13,  13,  13,  13,  12,  12,  12,
         12,  12,  12,  11,  11,  11,  11,  11,  11,  10,  10,  10,
         10,  10,  10,   9,   9,   9,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   8,   8]) 



In [11]:
cap_lens

tensor([ 15,  15,  15,  15,  13,  13,  13,  13,  13,  12,  12,  12,
         12,  12,  12,  11,  11,  11,  11,  11,  11,  10,  10,  10,
         10,  10,  10,   9,   9,   9,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   8,   8])

In [12]:
hidden

(tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]]),
 tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0

In [16]:
# source coreml/bin/activate
import coremltools

# 